In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import os

## Carregamento dos dados
O gráfico foi importado via biblioteca pandas, com separador por tabulação.

Usei o skiprows para ignorar a primeira linha e remover o "# Constructed from biom file".

In [38]:
df = pd.read_csv("tabela.tsv", sep="\t", skiprows=1)

In [39]:
df.head(3)

,#OTU ID,16S-Amostra1,16S-Amostra2,taxonomy
0,321501a64794e926be7b961674c35a0f,0.0,1170.0,d__Bacteria; p__Actinobacteriota; c__Actinomyc...
1,2c47537f334048fd1472da325f00561d,0.0,316.0,d__Bacteria; p__Actinobacteriota; c__Actinomyc...
2,4a1547af0bbae2aff8f3222d2fba2102,179.0,0.0,d__Bacteria; p__Firmicutes_A; c__Clostridia; o...


## ----------------------------------------------------------

## Abundância Relativa por amostra

O gênero foi obtido da coluna "taxonomy", sendo aqueles que se inicial por "g__"

In [40]:
df["Genero"] = df["taxonomy"].str.extract(r"g__([^;]*)")

Definindo valores sem informações de gênero como "Desconhecido"

In [41]:
df["Genero"] = df["Genero"].fillna("Desconhecido")

Optei por remover os sufixos "_A", "_B" por exemplo, dos gêneros para padronizar e evitar confusão. Supouz que ambos são do mesmos gênero e isso se deve a diferentes forams de representar alguma distintção dentro do gênero, mas sem uma classificação formal de espécie.

In [42]:
df["Genero"] = df["Genero"].str.replace(r"_\w$", "", regex=True)

Os dados quantitativos (provavelmente reads) de cada amostra foram somados e agrupados por gênero

In [43]:
genero_abundancia = df.groupby("Genero")[["16S-Amostra1", "16S-Amostra2"]].sum()

A abundância relativa foi calculada através do percentual quantitativo de cada gênero em relação ao total. Optei por arredondar para 2 casas decimais visando facilitar a visualização

In [44]:
genero_abundancia_relativa = (genero_abundancia.div(genero_abundancia.sum()) * 100).round(2)
genero_abundancia_relativa.columns = ["Amostra 1 (%)", "Amostra 2 (%)"]
genero_abundancia_relativa = genero_abundancia_relativa.reset_index() 

pd.set_option('display.max_rows', None) #OBS. Esse comando permite desabilitar o número máximo de linhas mostradas
genero_abundancia_relativa


,Genero,Amostra 1 (%),Amostra 2 (%)
0,Acetatifactor,0.80,0.00
1,Actinopolyspora,0.00,1.17
2,Agathobacter,3.35,0.00
3,Agathobaculum,1.48,0.00
4,Akkermansia,0.19,0.00
5,Alicyclobacillus,0.00,0.21
6,Alistipes,0.95,0.00
7,Anaerobutyricum,0.49,0.00
8,Anaerostipes,1.33,0.00
9,Bacteroides,4.72,0.00


A tabela foi salva em "genero_abundancia_relativa.tsv"

In [45]:
genero_abundancia_relativa.to_csv("genero_abundancia_relativa.tsv", sep="\t", index=False)

## ----------------------------------------------------------

## Gráfico de Barras Empilhadas

O seguinte código abaixo busca desenvolver um gráfico de barras empilhadas considerando o gênero e a abundância relativa calculada anteriormente para cada uma das amostras. Optou-se por fazer um gráfico interativo para melhor visuzalização de cada gênero e sua respectiva abundância de forma individual.

In [83]:
amostras = ["Amostra 01", "Amostra 02"]

cores = px.colors.qualitative.Plotly

trace_list = []
for i, genero in enumerate(genero_abundancia_relativa["Genero"].unique()):
    valores = genero_abundancia_relativa.loc[
        genero_abundancia_relativa["Genero"] == genero, ["Amostra 1 (%)", "Amostra 2 (%)"]
    ].values.flatten()

    trace = go.Bar(
        x=amostras,
        y=valores,
        name=genero, 
        hovertemplate='<b>%{x}</b><br>Gênero: ' + genero + '<br>Abundância Relativa: %{y:.2f}%',
        marker=dict(color=cores[i % len(cores)], line=dict(color="black", width=0.7)),
        hoverlabel=dict(namelength=0) 
    )
    trace_list.append(trace)

layout = go.Layout(
    title=dict(
        text="Gráfico de barras empilhadas sobre a Abundância Relativa de Gêneros nas Amostras",
        x=0.5, 
        font=dict(size=16, family="Arial, sans-serif", color="black")
    ),
    xaxis=dict(
        title="Amostras",
        tickfont=dict(size=14),
        showline=True,
        linecolor="black"
    ),
    yaxis=dict(
        title="Abundância Relativa (%)",
        tickfont=dict(size=14),
        showgrid=True,
        gridcolor="lightgrey",
        zeroline=True,
        zerolinecolor="black"
    ),
    barmode="stack",
    legend=dict(
        title="Gêneros",
        x=1.02,
        y=1,
        font=dict(size=12)
    ),
    plot_bgcolor="white",
    margin=dict(t=60, r=50, b=60, l=70),
)

fig = go.Figure(data=trace_list, layout=layout)
fig.show()

## ----------------------------------------------------------

## Outras sugestões de gráficos

## Heatmap

In [84]:
data = genero_abundancia_relativa[['Genero', 'Amostra 1 (%)', 'Amostra 2 (%)']].set_index('Genero').transpose()

fig2 = go.Figure(data=go.Heatmap(
    z=data.values,
    x=data.columns,
    y=data.index,
    colorscale='Blues',
    colorbar=dict(title="Abundância Relativa (%)"),
    text=data.values,
    hoverinfo='text',
    showscale=True,
    hovertemplate='<br>Gênero: %{x} ' + '<br>Abundância Relativa: %{z:.2f}%',
    hoverlabel=dict(namelength=0)
))

fig2.update_layout(
    title='Heatmap sobre a Abundância Relativa de Gêneros nas Amostras',
    xaxis_title='Gênero',
    yaxis_title='Amostras',
    xaxis=dict(showgrid=True, tickangle=45),
    yaxis=dict(tickvals=[0, 1], ticktext=['Amostra 1 (%)', 'Amostra 2 (%)']),
    height=300, 
    margin=dict(l=50, r=50, t=50, b=100), 
    xaxis_tickangle=45,
)

fig2.show()



## Criação de um relatório automatizado

Gerar tabela em HTML formatada

In [73]:
tabela_html = genero_abundancia_relativa.to_html(index=False, classes='table table-striped table-responsive text-center', border=0)

Criar relatório em HTML

In [89]:
html_content = f'''
<!DOCTYPE html>
<html lang="pt-br">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Processo seletivo - Bioinformática GoGenetic (Relatório)</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/bootstrap/5.3.0/css/bootstrap.min.css">
    <style>
        .container {{
            max-width: 800px;
            margin: auto;
        }}
        table {{
            width: 100%;
            text-align: center;
        }}
    </style>
</head>
<body>
    <div class="container">
        <h1 class="text-center mt-4">Processo seletivo - Bioinformática GoGenetic (Relatório)</h1>
        <hr>
        <h2 class="text-center">Tabela de Abundância Relativa</h2>
        {tabela_html}
        <hr>
        <h2 class="text-center">Gráficos de Distribuição</h2>
        <div id="grafico"></div>
        <hr>
        <h2 class="text-center"></h2>
        <div id="grafico2"></div>
    </div>
    <script>
        var grafico = {fig.to_json()};
        Plotly.newPlot('grafico', grafico.data, grafico.layout);

        var grafico2 = {fig2.to_json()};
        Plotly.newPlot('grafico2', grafico2.data, grafico2.layout);
    </script>
</body>
</html>
'''

Salvar o relatório como um arquivo HTML

In [90]:
report_path = "relatorio_interativo.html"
with open(report_path, "w", encoding="utf-8") as f:
    f.write(html_content)